In [1]:
!pip show Manim

Name: manim
Version: 0.19.0
Summary: Animation engine for explanatory math videos.
Home-page: 
Author: The Manim Community Developers
Author-email: contact@manim.community
License: MIT
Location: c:\users\zachl\appdata\local\programs\python\python310\lib\site-packages
Requires: av, beautifulsoup4, click, cloup, decorator, isosurfaces, manimpango, mapbox-earcut, moderngl, moderngl-window, networkx, numpy, Pillow, pycairo, pydub, Pygments, rich, scipy, screeninfo, skia-pathops, srt, svgelements, tqdm, typing-extensions, watchdog
Required-by: 


In [2]:
from manim import *
import numpy as np
import scipy.stats as stats
config.media_width = "75%"
config.verbosity = "WARNING"

In [3]:
%%manim -qm -p FitGaussian3D

class FitGaussian3D(ThreeDScene):
    def construct(self):
        # Step 1: Create 3D Axes
        axes = ThreeDAxes(
            x_range=[-3, 3, 1],
            y_range=[-3, 3, 1],
            z_range=[0, 0.25, 0.25],
            axis_config={"color": BLUE}
        )

        # Step 2: Sample 2D Gaussian Data Points
        mean = np.array([0, 0])  
        cov = np.array([[0.25, 0], [0, 0.25]])  
        samples = np.random.multivariate_normal(mean, cov, 100)

        dots = VGroup(*[
            Dot3D(axes.c2p(x, y, 0), color=RED, radius=0.05) for x, y in samples
        ])

        # Step 3: Create the 3D Gaussian Surface
        def gaussian_surface(u, v):
            pos = np.array([u, v])
            pdf = stats.multivariate_normal(mean=mean, cov=cov).pdf(pos)
            return [u, v, pdf * 3]  

        gaussian = Surface(
            lambda u, v: np.array(gaussian_surface(u, v)),
            u_range=[-1.5, 1.5], v_range=[-1.5, 1.5],
            resolution=(30, 30),
            color=YELLOW,
            fill_opacity=0.75
        )

        # 🔹 Step 4: Set Initial Camera Zoomed Out
        self.set_camera_orientation(phi=0 * DEGREES, theta=-90 * DEGREES, gamma=0*DEGREES, zoom=0.6)

        # Animations
        self.play(Create(axes))
        self.wait()
        self.play(FadeIn(dots))
        self.wait()

        # # 🔹 Step 5: Rotate Camera as Gaussian is Drawn
        self.begin_ambient_camera_rotation(rate=60/6 * DEGREES, about="phi")
        self.begin_ambient_camera_rotation(rate=45/6 * DEGREES, about="theta")
    
        self.wait()
        self.play(Create(gaussian), run_time=3)
        self.move_camera(
                zoom=1.0,  # Zoom in
                run_time=2,  # Time for this animation
            )
        
        self.stop_ambient_camera_rotation(about='phi')
        self.stop_ambient_camera_rotation(about='theta')

        self.wait()


Manim Community v0.19.0

In [ ]:
%%manim -qm MaximumLikelihoodDemo

import numpy as np
import scipy.stats as stats
from manim import *

class MaximumLikelihoodDemo(Scene):
    def construct(self):
        # Define the axes
        axes = Axes(
            x_range=[0, 10, 1],
            y_range=[0, 1, 0.1],
            tips=False,
            axis_config={"include_numbers": True}
        )

        # Sample from a Gaussian
        samples = np.random.normal(5, 1, 20)

        # Perform MLE to estimate mean and std
        mu, sigma = stats.norm.fit(samples)

        # Generate x values for the estimated Gaussian
        x_vals = np.linspace(0, 10, 100)
        y_vals = stats.norm.pdf(x_vals, mu, sigma)  # Compute Gaussian PDF

        # Normalize to fit within y-axis range
        y_vals /= np.max(y_vals)  # Scale between 0 and 1

        # Create the probability density function (PDF) curve
        pdf_curve = axes.plot(lambda x: stats.norm.pdf(x, mu, sigma) / np.max(y_vals), color=BLUE)

        # Create the sample dots
        dots = VGroup(*[
            Dot(axes.c2p(x, 0), color=RED, radius=0.05) for x in samples
        ])

        # Animate
        self.play(Create(axes))
        self.wait()
        self.play(FadeIn(dots))
        self.wait()
        self.play(Create(pdf_curve))  # Animate the fitted Gaussian
        self.wait()


Manim Community v0.19.0